# Transfer Learning wwith TensorFlow part 1: Feature Extraction

Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problem.

There are two main genefits:

**1.** Can leverage an existing neural network architecture proven to work on problems similar to our own.

**2.** Can levarage a working neural network architecture which has already learned patterns on similar data to our own, then we can adapt those patterns to our data.

In [17]:
# Import dependencies
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import datetime

from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Downloading and becoming one with the data
- Get data (10% of 10 food classes from Food101)

## Creating data loaders (preparing the data)

We will use `ImageDataGenerator` class to load in our images in batches.

In [8]:
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training image")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size = IMAGE_SHAPE,
                                                          batch_size = BATCH_SIZE,
                                                          class_mode = "categorical")

print("Testing image")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size = IMAGE_SHAPE,
                                             batch_size = BATCH_SIZE,
                                              class_mode = "categorical")

Training image
Found 750 images belonging to 10 classes.
Testing image
Found 2500 images belonging to 10 classes.


## Setting up callbacks (things to run whilst or model trains)

Callbacks are extra functionality you can add to your model to be performed during or after training. Some of the mosr popular callbacks are:

**1.** Tracking experience with the `TensorBoard` callback

**2.** Model checkpoint with the `ModelCheckPoint` callback

**3.** Stooping a model from training (before it trains too long and overfits) with the `EarlyStopping` cllback


In [12]:
# Create TensorBoard callback and functionized

def create_TensorBoard_callback(dir_name, experiment_name):

    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard lod files to: {log_dir}")
    
    return tensorboard_callback

> This is a TensorFlow callback TensorBoard function. 🤖

## Creating model using TensorFlow Hub

In the past we used TensorFlow to create our own modles layer by layer from scratch.

Now we are going to do similar procedd, expect the majroity of our model's layers are goind to come from TensorFlow Hub.

We can access pre-traied models on: [TfHub.dev](https://www.kaggle.com/models?tfhub-redirect=true)

In [15]:
# Let's compare the following two models
resnet_url = "https://kaggle.com/models/google/resnet-v2/frameworks/TensorFlow2/variations/50-feature-vector/versions/1"

efficientnet_url = "https://www.kaggle.com/models/tensorflow/efficientnet/frameworks/TensorFlow2/variations/b0-feature-vector/versions/1"

In [18]:
# Let's make a create_model() function to create a model from URL

def create_model(modle_URL, num_classes=10):

    """
    Take a TensorFlow Hub URL and create a Keras Sequential model with it.
    """

    # Download the pretrained model and save it as a Keras layer
    feature_extractor_layer = hub.KerasLayer(modle_URL,
                                             trainable=False, # Freeze the already patterns
                                             name="Feature_Extractions",
                                             input_shape=IMAGE_SHAPE+(3,))
    
    # Create our own model
    model = tf.keras.Sequential([
        feature_extractor_layer,
        tf.keras.layers.Dense(num_classes, activation='softmax', name="output_layer")
    ])

    # Return
    return model

### Create ResNet TensorFlow Hub Feature Extraction Model 👻

In [19]:
# Create resnet model
resnet_model = create_model(resnet_url, num_classes=train_data_10_percent.num_classes)

In [20]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Feature_Extractions (KerasL  (None, 2048)             23564800  
 ayer)                                                           
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


> Because we set trainable=`False`. Hence Non-trainable params: 23,564,800.

In [25]:
# Complie a model
resnet_model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.legacy.Adam(),
    metrics = ["accuracy"]
)

# Fit the model
resnet_history = resnet_model.fit(train_data_10_percent,
                 steps_per_epoch=len(train_data_10_percent),
                 epochs=5,
                 validation_data=test_data,
                 callbacks=[create_TensorBoard_callback("tensorflow_hub",
                                                        experiment_name="resnet50V2")]
                 )

Saving TensorBoard lod files to: tensorflow_hub/resnet50V2/20240204-235606
Epoch 1/5


2024-02-04 23:56:07.081375: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


24/24 [==============================] - 53s 2s/step - loss: 0.6972 - accuracy: 0.7893 - val_loss: 0.7544 - val_accuracy: 0.7524
Epoch 2/5
24/24 [==============================] - 53s 2s/step - loss: 0.4582 - accuracy: 0.8747 - val_loss: 0.6858 - val_accuracy: 0.7764
Epoch 3/5
24/24 [==============================] - 53s 2s/step - loss: 0.3370 - accuracy: 0.9280 - val_loss: 0.6568 - val_accuracy: 0.7872
Epoch 4/5
24/24 [==============================] - 53s 2s/step - loss: 0.2673 - accuracy: 0.9520 - val_loss: 0.6467 - val_accuracy: 0.7880
Epoch 5/5
24/24 [==============================] - 53s 2s/step - loss: 0.2112 - accuracy: 0.9693 - val_loss: 0.6383 - val_accuracy: 0.7932
